In [ ]:
import ee
import numpy as np
#from pprint import pprint

ee.Initialize()
print('EE API version: ',ee.__version__)

### Import collections

In [ ]:
## Counties
regions = ee.FeatureCollection('users/lizcanosandoval/wv-regions/wv_counties_buffer')

## Import DEM collection:
dem = ee.ImageCollection("projects/imars-3d-wetlands/DEM_GoM")

## Import image collection:
wetlands = ee.ImageCollection("projects/imars-3d-wetlands/alabama")\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_big_bend"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_monroe"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_ne"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_panhandle"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_se"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_sw"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/louisiana_e"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/louisiana_w"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/mississippi"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/texas_central"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/texas_ne"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/texas_sw"))

### Check 'regions' properties and prepare metadata

In [ ]:
## Check State names
print(regions.aggregate_array('1_State').distinct().getInfo())

In [ ]:
## Check County names
stateLabel = 'Florida'
state = regions.filterMetadata('1_State','equals',stateLabel)
print(state.aggregate_array('2_County').distinct().sort().getInfo())

In [ ]:
## List with County names
countyList = [
'Brevard',
'Broward',
'Charlotte',
'Citrus',
'Clay',
'Collier',
'Desoto',
'Dixie',
'Duval',
'Escambia',
'Flagler',
'Franklin',
'Gulf',
'Hernando',
'Hillsborough',
'Indian River',
'Jefferson',
'Lee',
'Levy',
'Liberty',
'Manatee',
'Martin',
'Miami-Dade',
'Monroe',
'Nassau',
'Okaloosa',
'Palm Beach',
'Pasco',
'Pinellas',
'Putnam',
'Saint Johns',
'Saint Lucie',
'Santa Rosa',
'Sarasota',
'Taylor',
'Volusia',
'Wakulla',
'Walton']
print('Counties in list:', len(countyList))

In [ ]:
## Set elevation ranges at x steps
minRange = 0
maxRange = 3
step = 0.1

# Create list of numbers
demRanges = np.arange(minRange,maxRange+step,step).tolist()
# Reformat numbers
demRanges = [round(num,1) for num in demRanges]
print(demRanges)

In [ ]:
for c in list(range(len(countyList))):
    
    ## Select county, and set time and end months to filter collection
    countyLabel = countyList[c]
    county = state.filterMetadata('2_County','equals',countyLabel)
    time_start = 7
    time_end = 12
    
    ## Filter collection
    filterCollection = wetlands.filterBounds(county) \
    .filter(ee.Filter.calendarRange(time_start,time_end,'month'))
    #.filterDate(time_start,time_end)
    
    
    ### CLEAN MISCLASSIFIED PIXELS
    ## Function to select pixels equal to 11-developed
    def developed(image):
        mask = ee.Image(image).eq(11)
        return image.updateMask(mask)

    ## Function to create mask of only water
    def water(image):
        mask = ee.Image(image).eq(3)
        return image.updateMask(mask)

    ## Function to clean missclassified pixels
    def cleanDeveloped(collection):
        # Clean misclassified cloudy pixels:
        fullCollection = collection.filterBounds(county)
        # Select only developed pixels
        devCollection = fullCollection.map(developed)
        # Create mosaic suming pixel values
        sumDeveloped = devCollection.sum()
        # Select pixels greater than 33 (presence of developed class in 3 or more images)
        selectThreshold = sumDeveloped.gt(32)
        # Create a mask for misclassified pixels
        justDeveloped = sumDeveloped.updateMask(selectThreshold)
        # Some areas over water bodies have cloudy pixels we can mask out
        waterCollection = fullCollection.map(water)
        waterMask = ee.ImageCollection(waterCollection).max().mask().Not()
        return justDeveloped.updateMask(waterMask)

    ## Apply water mask on the cleaned collection. This is the mask for "real" developed pixels.
    realDeveloped = cleanDeveloped(wetlands)
    
    
    ### FINAL MOSAIC
    ## Function to clean mosaic
    def clean_function(image):
        developed = image.eq(11) #Developed
        imageUpdated = image.updateMask(developed) #Developed mask
        falseImg = realDeveloped.mask().Not()
        maskDev = imageUpdated.updateMask(falseImg).mask().Not()
        imageMask = image.updateMask(maskDev)
        return imageMask

    cleanCollection = filterCollection.map(clean_function)

    ## Mosaic
    mosaicImage = ee.ImageCollection(cleanCollection).max().clip(county)
    
    
    ### CLIP DEM
    ## Clip DEM by counties selected:
    ## Filter the collection by dates and counties selected.
    filterDEM = dem.filterBounds(county);
    clipDEM = filterDEM.mosaic().clip(county);

    
    ### FUNCTION TO CALCULATE AREAS
    ## Geometry used for exporting arguments
    geometry = ee.Geometry(county.geometry(10)).bounds()

    ## Function to calculate areas
    def calculateAreas(mosaic):
        areaImage = ee.Image.pixelArea().addBands(mosaic)

        ## Get total area (m2) per class:
        areas = areaImage.reduceRegion(
            reducer= ee.Reducer.sum().group(
                groupField= 1,
                groupName= 'code'),
            geometry= geometry,
            scale= 2,
            maxPixels= 1e13
            )
        ## ee.List:
        classes = [{'class':'Cloud'},{'class':'Soil'},{'class':'Water'},{'class':'Dead grass'},{'class':'Marsh'},{'class':'Scrub'},
                  {'class':'Grass'},{'class':'Forested upland'},{'class':'Forested wetland'},{'class':'Developed'}]
        classAreas = (ee.List(areas.get('groups')).zip(classes)) 

        ## Convert m^2 to km^2:
        def area_list(item):
            #areaDict = ee.Dictionary(item)
            dict1 = ee.Dictionary(ee.List(item).get(0))
            dict2 = ee.Dictionary(ee.List(item).get(1))
            areaDict = dict1.combine(dict2)
            CODE = ee.Number(areaDict.get('code')).format()
            AREA = ee.Number(areaDict.get('sum')).divide(1e6)
            CLASS = areaDict.get('class')

            feature = ee.Feature(None,{'state':stateLabel,'county':countyLabel,'code':CODE,'class':CLASS,'area_km2':AREA})
            return feature

        featureCollection = ee.FeatureCollection(classAreas.map(area_list))
        return featureCollection
    
    
    ### CHANGE PIXEL VALUES
    ## Reproject mosaic and remap pixel values
    fr = [0,1,2,3,5,6,7,8,9,10,11]
    to = [0,1,2,3,4,5,6,7,8,9,10]
    mosaic_repr = mosaicImage.reproject(
        crs= 'EPSG:4326',
        scale= 2).remap(fr,to)

    
    ### EXPORT AND SUBMIT TASKS
    ## Export areas per elevation ranges
    for i in list(range(len(demRanges)))[0:-1]:
        minElev = demRanges[i]
        maxElev = demRanges[i+1]

        # Function to mask ranges
        def maskRange(image,min,max):
            mask = ee.Image(image).gte(min).And(ee.Image(image).lt(max))
            return image.updateMask(mask)

        # Get DEM and Mosaic ranges
        demRange = maskRange(clipDEM,minElev,maxElev).mask()
        mosaicRange = mosaic_repr.updateMask(demRange)

        # Estimate areas
        areasFile = calculateAreas(mosaicRange)

        # File name
        fileName = stateLabel+'_'+countyLabel+'_'+str(minElev)+'-'+str(maxElev)+'_m'

        ## export
        task = ee.batch.Export.table.toDrive(\
              collection= areasFile,
              description= fileName,
              folder= 'Wetlands',
              fileNamePrefix= fileName,
              fileFormat= 'CSV')

        task.start()
        print('     Submitted '+fileName)
    print('Export completed for '+countyLabel+' County')
print('ALL TASKS SUBMITTED!')